In [ ]:
from LODEGP.LODEGP import LODEGP, list_standard_models
import gpytorch
import torch
import matplotlib.pyplot as plt
from finite_difference import central_difference
from helpers.training_functions import granso_optimization
from helpers.util_functions import get_full_kernels_in_kernel_expression
import numpy as np

In [ ]:
torch.set_default_dtype(torch.float64)

In [ ]:
# Base ODE:     
# [x**2 + 981/(100*l1), 0, -1/l1, 0, x**2+981/(100*l2), -1/l2]

In [ ]:
START = 2r
END = 12r
COUNT = 100r
train_x = torch.linspace(START, END, COUNT)
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=5r)

y0_func = lambda x: float(781/8000)*torch.sin(x)/x - float(1/20)*torch.cos(x)/x**2 + float(1/20)*torch.sin(x)/x**3
y1_func = lambda x: float(881/8000)*torch.sin(x)/x - float(1/40)*torch.cos(x)/x**2 + float(1/40)*torch.sin(x)/x**3
y2_func = lambda x: float(688061/800000)*torch.sin(x)/x - float(2543/4000)*torch.cos(x)/x**2 + float(1743/4000)*torch.sin(x)/x**3 - float(3/5)*torch.cos(x)/x**4 + float(3/5)*torch.sin(x)/x**5 
y0 = y0_func(train_x)
y1 = y1_func(train_x)
y2 = y2_func(train_x)
train_y = torch.stack([y0, y1, y2], dim=-1r)

model = LODEGP(train_x, train_y, likelihood, 3, ODE_name="Bipendulum", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})
#model = LODEGP(train_x, train_y, likelihood, 5, ODE_name="Heating", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})

In [ ]:
model.matrix_multiplication
# This should satisfy the ODEs per column?
model.diffed_kernel 
model.sum_diff_replaced[2][0]
#str(model.covar_description[0][0])[47:-10]
#model.covar_description
#model.covar_module 

In [ ]:
var("signal_variance_1, t_diff, lengthscale_1, signal_variance_2, lengthscale_2, t1, t2") 

k_00 = 962361/40000*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2) - 9.81*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^2 + 9.81*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^4 + 3.0*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^4 - 6.0*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^6 + 1.0*signal_variance_2^2*t_diff^4*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^8
k_10 = 962361/40000*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2) - 7.3575*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^2 + 7.3575*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^4 + 1.5*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^4 - 3.0*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^6 + 0.5*signal_variance_2^2*t_diff^4*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^8
k_20 = 944076141/4000000*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2) - 120.295125*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^2 + 120.295125*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^4 + 58.86*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^4 - 117.72*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^6 + 19.62*signal_variance_2^2*t_diff^4*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^8 - 15.0*signal_variance_2^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^6 + 45.0*signal_variance_2^2*t_diff^2*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^8 - 15.0*signal_variance_2^2*t_diff^4*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^10 + 1.0*signal_variance_2^2*t_diff^6*e^(-0.5*t_diff^2/lengthscale_2^2)/lengthscale_2^12
k_00 = k_00.substitute(t_diff==t1-t2)
k_10 = k_10.substitute(t_diff==t1-t2)
k_20 = k_20.substitute(t_diff==t1-t2)

In [ ]:
#print((f0.diff(t, 2) + 9.81*f0 - f2))
#print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))
(k_00.diff(t1, 2) + 9.81*k_00 - k_20)
k_10.diff(t1, 2) + 9.81/2*k_10 - k_20/2

In [ ]:
# If I want to extract the string of the function
#str(model.covar_description[0][0])[47:-10]

def covar_description_function00(x1, x2, row_pos=0, col_pos=0):
    common_terms = {
                "t_diff" : x1-x2.t(),
                "t_sum" : x1+x2.t(),
                "t_ones": torch.ones_like(x1-x2.t()),
                "t_zeroes": torch.zeros_like(x1-x2.t())
            }
    model_parameters = model.model_parameters
    return eval(model.covar_description[row_pos][col_pos]).detach().numpy()


h = 1e-4r
target_column = 2
for value in torch.linspace(0.0r, 1.0r, 1r):
    print(value)
    #print((f0.diff(t, 2) + 9.81*f0 - f2))
    #print((central_difference(covar_description_function00, value, h=1e-1r, precision=6, order=2) + 9.81*covar_description_function00(value, 0, 0) - covar_description_function00(value, 2, 0)))
    print((covar_description_function00(value+h, value, 0, target_column) - 2 * covar_description_function00(value, value, 0, target_column) + covar_description_function00(value-h, value, 0, target_column))/(h**2) + 9.81*covar_description_function00(value, value, 0, target_column) -  covar_description_function00(value, value, 2, target_column))
    


    #print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))
    print((covar_description_function00(value+h, value, 1, target_column) - 2 * covar_description_function00(value, value, 1, target_column) + covar_description_function00(value-h, value, 1, target_column))/(h**2) + 9.81/2*covar_description_function00(value, value, 1, target_column) -  covar_description_function00(value, value, 2, target_column)/2)

In [ ]:
# Comparison whether the central difference of the eval kernel is to the computer algebra derivative

#print((f0.diff(t, 2) + 9.81*f0 - f2))
#print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))
var("t1, t2")

row_pos = 0
col_pos = 0


def covar_description_function00(x1, x2, row_pos=0, col_pos=0):
    common_terms = {
                "t_diff" : x1-x2.t(),
                "t_sum" : x1+x2.t(),
                "t_ones": torch.ones_like(x1-x2.t()),
                "t_zeroes": torch.zeros_like(x1-x2.t())
            }
    model_parameters = model.model_parameters
    return eval(model.covar_description[row_pos][col_pos]).detach().numpy()

h = 1e-4r 

for value in torch.linspace(0.0r, 1.0r, 1r):
    compalg_val = model.diffed_kernel[row_pos][col_pos].diff(t1, 2)(t1=float(value), t2=float(value), signal_variance_2=1.0, lengthscale_2=1.0)
    fin_diff_torch_val = (covar_description_function00(value+h, value) - 2 * covar_description_function00(value, value) + covar_description_function00(value-h, value))/(h**2)
    print(f"compalg_val: {compalg_val}, fin_diff_torch_val: {fin_diff_torch_val}")
    

In [ ]:
model.covar_description[0][0]

In [ ]:
# Comparison whether the computer algebra function does the same as the torch eval function
#print((f0.diff(t, 2) + 9.81*f0 - f2))
#print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))
var("t1, t2")


row_pos = 0
col_pos = 0
for value in torch.linspace(0.0r, 1.0r, 1r):
    compalg_val = model.diffed_kernel[row_pos][col_pos](t1=float(value), t2=float(value), signal_variance_2=1.0, lengthscale_2=1.0)
    fin_diff_torch_val = covar_description_function00(value, value, row_pos, col_pos)
    print(f"compalg_val: {compalg_val}, fin_diff_torch_val: {fin_diff_torch_val}")
    

In [ ]:
# Column wise satisfaction of the ODEs through sagemath entries within the LODEGP 
target_col = 0
diffed_kernel_column = [model.diffed_kernel[row][target_col] for row in range(model.num_tasks)]

#print((f0.diff(t, 2) + 9.81*f0 - f2))
#print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))
var("t1")

print((diffed_kernel_column[0].diff(t1, 2) + 9.81*diffed_kernel_column[0] - diffed_kernel_column[2])(t1=1, t2=1, signal_variance_2=1.0, lengthscale_2=1.0))
(diffed_kernel_column[1].diff(t1, 2) + 9.81/2*diffed_kernel_column[1] - diffed_kernel_column[2]/2)(t1=1, t2=1, signal_variance_2=1.0, lengthscale_2=1.0)

In [ ]:
V = model.V
var("t")
base_func = 1/(40*t)*sin(t)
p = matrix([[0],[0],[base_func]])
f = V*p
print(f)

f0 = 1/200*(200*base_func.diff(t, 2) + 981*base_func)
f1 = 1/200*(100*base_func.diff(t, 2) + 981*base_func)
f2 = 1/20000*(20000*base_func.diff(t, 4) + 294300*base_func.diff(t, 2) + 981**2*base_func)
plot(f1, (t, -10, 10), color='red')
f1

In [ ]:
# Check if both ODEs are satisfied
print((f0.diff(t, 2) + 9.81*f0 - f2))
print((f1.diff(t, 2) + 9.81/2*f1 - f2/2))

In [ ]:
values = list()
differences = list()
for value in train_x:
    cd_val = float(central_difference(y0_func, value, h=1e-1r, precision=6, order=2))
    compalg_val = f0.diff(t, 2)(t=float(value)).n()
    values.append((cd_val, compalg_val))
    differences.append(abs(cd_val - compalg_val))
differences

In [ ]:
# ODE 1
#(f0.diff(t, 2) + 9.81*f0 - f2)
h = 1e-1r
print("ODE1 median error: \t" , np.median(np.abs(central_difference(y0_func, train_x, h=h, precision=6, order=2) + 9.81r*y0_func(train_x) - y2_func(train_x))))
print("ODE1 avg. error: \t" , np.average(np.abs(central_difference(y0_func, train_x, h=h, precision=6, order=2) + 9.81r*y0_func(train_x) - y2_func(train_x))))



# ODE 2
#(f1.diff(t, 2) + 9.81/2*f1 - f2/2)

print("ODE2 median error: \t" , np.median(np.abs(central_difference(y1_func, train_x, h=h, precision=6, order=2) + 9.81r*y1_func(train_x)*0.5r - y2_func(train_x)*0.5r)))
print("ODE2 avg. error: \t" , np.average(np.abs(central_difference(y1_func, train_x, h=h, precision=6, order=2) + 9.81r*y1_func(train_x)*0.5r - y2_func(train_x)*0.5r)))

In [ ]:
# Plot the posterior GP and the data
model.eval()
model.likelihood.eval()
with torch.no_grad():
    test_x = torch.linspace(2r, 12r, 100r)
    observed_pred = model.likelihood(model(test_x))
    observed_pred_mean = observed_pred.mean
    observed_pred_var = observed_pred.covariance_matrix.diag().reshape(-1, 3)
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    for i in range( 3):
        ax.plot(test_x.numpy(), observed_pred_mean[:, i].numpy(), 'k')
        ax.fill_between(test_x.numpy(),
                        observed_pred_mean[:, i].numpy() - 1.96 * observed_pred_var[:, i].sqrt().numpy(),
                        observed_pred_mean[:, i].numpy() + 1.96 * observed_pred_var[:, i].sqrt().numpy(),
                        alpha=0.5)
        ax.scatter(train_x.numpy(), train_y[:, i].numpy(), s=10, c='r', marker='x')
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
    ax.set_title('Posterior GP')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()

In [ ]:
def get_param_spec(
    param_name,
    kernel_name,
    param_specs=None,
    kernel_param_specs=None,
    default_spec=None
):
    """
    Resolves a parameter spec using full name match, kernel+param, or fuzzy match for LODE_Kernel.

    Returns:
    - spec dict (e.g., {"bounds": (a, b), "type": ..., "mean": ..., "variance": ...})
    """
    param_specs = param_specs or {}
    kernel_param_specs = kernel_param_specs or {}
    default_spec = default_spec or {}

    # 1. Full name override
    if param_name in param_specs:
        return param_specs[param_name]

    # 2. Kernel + parameter
    if (kernel_name, param_name) in kernel_param_specs:
        return kernel_param_specs[(kernel_name, param_name)]

    # 3. Fuzzy base-key match for LODE_Kernel
    if kernel_name == "LODE_Kernel":
        for base_key in ["lengthscale", "signal_variance"]:
            if base_key in param_name:
                if (kernel_name, base_key) in kernel_param_specs:
                    return kernel_param_specs[(kernel_name, base_key)]

    # 4. Fallback
    return default_spec

def collect_hyperparameter_prior_specs(
    model,
    param_specs=None,
    kernel_param_specs=None,
    default_mean=0.0,
    default_variance=1.0
):
    """
    Collects prior parameters (mean, variance) for all named hyperparameters in the model.

    Returns:
    - List of tuples: (param_name, tensor, mean, variance)
    """
    results = []
    kernel_types = get_full_kernels_in_kernel_expression(model.covar_module)
    kernel_index = 0

    for name, param in model.named_hyperparameters():
        param_tensor = param
        local_param_name = name.split(".")[-1]

        if name in param_specs:
            kernel_type = "<explicit>"
        elif "LODE_Kernel" in kernel_types and "covar_module" in name:
            kernel_type = "LODE_Kernel"
        else:
            kernel_type = kernel_types[kernel_index] if kernel_index < len(kernel_types) else "<unknown>"
            kernel_index += 1

        spec = get_param_spec(
            name,
            kernel_type,
            param_specs=param_specs,
            kernel_param_specs=kernel_param_specs,
            default_spec={"mean": default_mean, "variance": default_variance}
        )

        results.append((name, param_tensor, spec["mean"], spec["variance"]))

    return results

kernel_param_specs = {
    ("RBFKernel", "lengthscale"): {"mean": 0.0, "variance": 0.5},
    ("MaternKernel", "lengthscale"): {"mean": 1.0, "variance": 2.0},
    ("ScaleKernel", "outputscale"): {"mean": 0.5, "variance": 1.0},
    ("LODE_Kernel", "signal_variance"): {"mean": 0.1, "variance": 0.1}
}

param_specs = {
    "likelihood.noise": {"mean": -2.0, "variance": 0.25}
}

model = LODEGP(train_x, train_y, likelihood, 3, ODE_name="Bipendulum Parameterized", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})

prior_specs = collect_hyperparameter_prior_specs(
    model,
    param_specs=param_specs,
    kernel_param_specs=kernel_param_specs,
    default_mean=0.0,
    default_variance=1.0
)
print(prior_specs)
mean_list = [p_spec[2] for p_spec in prior_specs]
mean_list
var_list = [p_spec[3] for p_spec in prior_specs]
var_list